# 04 - Evaluate

## API tests

Here are 4 samples of real data that you can use later at your own pace, just add app.

##### Sample 1

In [90]:
! curl https://capstone-uk-police.herokuapp.com/should_search/ -H "Content-Type: application/json" -d '{"observation_id": "3d931c84-94d3-4a14-869c-b10fef26e9ad", "Type": "Person search", "Date": "2020-01-01T00:56:00+00:00", "Part of a policing operation": false, "Latitude": null, "Longitude": null, "Gender": "Male", "Age range": "18-24", "Officer-defined ethnicity": "White", "Legislation": "Misuse of Drugs Act 1971 (section 23)", "Object of search": "Controlled drugs", "station": "nottinghamshire"}'

{"outcome":true}


In [91]:
! curl https://capstone-uk-police.herokuapp.com/search_result/ -H "Content-Type: application/json" -d '{"observation_id": "3d931c84-94d3-4a14-869c-b10fef26e9ad", "outcome": false}'

{"observation_id":"3d931c84-94d3-4a14-869c-b10fef26e9ad","outcome":false,"predicted_outcome":true}


##### Sample 2

In [92]:
! curl https://capstone-uk-police.herokuapp.com/should_search/ -H "Content-Type: application/json" -d '{"observation_id": "c6d2de91-445e-40f7-9e5f-9d572f8bb232", "Type": "Person search", "Date": "2020-01-01T05:10:30+00:00", "Part of a policing operation": null, "Latitude": 51.512598, "Longitude": -0.084967, "Gender": "Male", "Age range": "10-17", "Officer-defined ethnicity": "Black", "Legislation": "Misuse of Drugs Act 1971 (section 23)", "Object of search": "Controlled drugs", "station": "city-of-london"}'

{"outcome":true}


In [93]:
! curl https://capstone-uk-police.herokuapp.com/search_result/ -H "Content-Type: application/json" -d '{"observation_id": "c6d2de91-445e-40f7-9e5f-9d572f8bb232", "outcome": false}'

{"observation_id":"c6d2de91-445e-40f7-9e5f-9d572f8bb232","outcome":false,"predicted_outcome":true}


##### Sample 3

In [94]:
! curl https://capstone-uk-police.herokuapp.com/should_search/ -H "Content-Type: application/json" -d '{"observation_id": "ccf22593-5402-41b0-9f3b-ce4bb69389c2", "Type": "Person and Vehicle search", "Date": "2020-01-02T07:41:52+00:00", "Part of a policing operation": null, "Latitude": 51.51265, "Longitude": -0.101452, "Gender": "Male", "Age range": "18-24", "Officer-defined ethnicity": "Black", "Legislation": "Misuse of Drugs Act 1971 (section 23)", "Object of search": "Controlled drugs", "station": "city-of-london"}'

{"outcome":true}


In [95]:
! curl https://capstone-uk-police.herokuapp.com/search_result/ -H "Content-Type: application/json" -d '{"observation_id": "ccf22593-5402-41b0-9f3b-ce4bb69389c2", "outcome": true}'

{"observation_id":"ccf22593-5402-41b0-9f3b-ce4bb69389c2","outcome":true,"predicted_outcome":true}


##### Sample 4

In [96]:
! curl https://capstone-uk-police.herokuapp.com/should_search/ -H "Content-Type: application/json" -d '{"observation_id": "0bcac709-1531-49e4-881e-e0f5e1c28e31", "Type": "Person and Vehicle search", "Date": "2020-01-02T10:19:13+00:00", "Part of a policing operation": null, "Latitude": null, "Longitude": null, "Gender": "Male", "Age range": "18-24", "Officer-defined ethnicity": "White", "Legislation": "Misuse of Drugs Act 1971 (section 23)", "Object of search": "Controlled drugs", "station": "city-of-london"}'

{"outcome":true}


In [97]:
! curl https://capstone-uk-police.herokuapp.com/search_result/ -H "Content-Type: application/json" -d '{"observation_id": "0bcac709-1531-49e4-881e-e0f5e1c28e31", "outcome": true}'

{"observation_id":"0bcac709-1531-49e4-881e-e0f5e1c28e31","outcome":true,"predicted_outcome":true}


## Local tests

In [86]:
### IMPORTS ###

import os
import joblib
import json
import pickle
import pandas as pd
from flask import Flask, request, jsonify
from peewee import (
    SqliteDatabase, PostgresqlDatabase, Model, IntegerField, BooleanField,
    FloatField, TextField, IntegrityError
)
from playhouse.shortcuts import model_to_dict
from playhouse.db_url import connect

import sys
sys.path.append('../')

import src
from src.modeling import (
    coordinate_dict,
    fill_coordinates_with_station_means,
    grid_to_category,
    extract_datetime_features)



### UNPICKLE THE PREVIOUSLY-TRAINED SKLEARN MODEL ###

with open('../deploy/columns.json') as fh:
    columns = json.load(fh)

with open('../deploy/dtypes.pickle', 'rb') as fh:
    dtypes = pickle.load(fh)

pipeline = joblib.load('../deploy/pipeline.pickle')

In [88]:
d2 = {"observation_id": "c6d2de91-445e-40f7-9e5f-9d572f8bb232",
       "Type": "Person search",
      "Date": "2020-01-01T05:10:30+00:00",
      "Part of a policing operation": None,
      "Latitude": 51.512598,
      "Longitude": -0.084967,
      "Gender": "Male",
      "Age range": "10-17",
      "Officer-defined ethnicity": "Black",
      "Legislation": "Misuse of Drugs Act 1971 (section 23)",
      "Object of search": "Controlled drugs",
      "station": "city-of-london"}

keys = ['Type',
            'Date',
            'Part of a policing operation',
            'Latitude',
            'Longitude',
            'Gender',
            'Age range',
            'Officer-defined ethnicity',
            'Legislation',
            'Object of search',
            'station']
myobs = pd.DataFrame([[d2[key] for key in keys]], index=[d2['observation_id']], columns=columns).astype(dtypes)
myobs

,type,date,operation,lat,long,sex,age,ethnicity_officer,legislation,search_target,station
c6d2de91-445e-40f7-9e5f-9d572f8bb232,Person search,2020-01-01 05:10:30+00:00,NaN,51.512598,-0.084967,Male,10-17,Black,Misuse of Drugs Act 1971 (section 23),Controlled drugs,city-of-london


In [89]:
pipeline.predict(myobs)

array([False])